### Importing Libraries and Loading the Dataset

This code block is responsible for importing the necessary libraries and loading the dataset in Parquet format. The libraries `pandas` and `numpy` are imported for data manipulation and numerical operations, respectively. The dataset, which is stored in Parquet format for optimized loading and performance, is then loaded into a DataFrame.

Finally, the first five rows of the dataset are displayed using the `head()` function to verify that the dataset has been loaded correctly and is ready for further processing.

In [23]:
# Importar bibliotecas
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import lightgbm as lgb
from sklearn.multioutput import MultiOutputRegressor
import joblib
import os
# Carregar o dataset Parquet
file_path = 'C:\\Users\grazi\\Documents\\NASASPACEAPPS2025\\NASA-SPACE-APPS-2025\\notebooks\\DATASETS\\clima_brasil_mensal\\clima_brasil_2000_2025.parquet'
data = pd.read_parquet(file_path)
# Verificar as primeiras linhas
data.head()


,time,lat,lon,T2M,QV2M,PS,SWGDN,PRECTOT
0,2007-01-01 00:30:00,-90.0,-180.000,245.154404,0.000395,69392.148438,371.832153,0.000002
1,2007-01-01 00:30:00,-90.0,-179.375,245.154404,0.000395,69392.148438,371.832153,0.000002
2,2007-01-01 00:30:00,-90.0,-178.750,245.154404,0.000395,69392.148438,371.832153,0.000002
3,2007-01-01 00:30:00,-90.0,-178.125,245.154404,0.000395,69392.148438,371.832153,0.000002
4,2007-01-01 00:30:00,-90.0,-177.500,245.154404,0.000395,69392.148438,371.832153,0.000002


### Data Preprocessing

This block of code handles the necessary preprocessing steps to prepare the dataset for model training and testing. Several transformations and data filtering steps are applied to ensure the dataset is clean and relevant for the task at hand:

1. **Remove unnecessary columns**: Columns that will not be useful for the model (such as 'day', 'hour', and 'week_day') are dropped.
2. **Filter data for Brazil**: Data points outside the geographic boundaries of Brazil are removed, ensuring that only relevant data is included.
3. **Convert time**: The "time" column is converted to the `datetime` format for easier manipulation and extraction of date-related features.
4. **Temperature conversion**: The temperature variable (`T2M`) is converted from Kelvin to Celsius for more intuitive interpretation.
5. **Humidity conversion**: The humidity (`QV2M`) is converted from decimal to percentage.
6. **Create new date-related variables**: Year and month columns are extracted from the "time" column, which will be useful for future predictions.
7. **Temperature range creation**: A margin of error is applied to the temperature data to create new variables for maximum and minimum temperature based on a 3-degree negative and 4-degree positive range from the average temperature (`T2M`).

Finally, the data is displayed after preprocessing to confirm the changes.


In [14]:
# Definir o número de segundos em um mês (30 dias)
seconds_in_month = 30 * 24 * 60 * 60

# Multiplicar PRECTOT pelo número de segundos em um mês
data['PRECTOT'] = (data['PRECTOT'] * seconds_in_month).round(2)

# Excluir as colunas 'day', 'hour' e 'week_day' 
colunas_para_excluir = ['day', 'hour', 'week_day']
colunas_existentes = [col for col in colunas_para_excluir if col in data.columns]

# Remover as colunas existentes
data = data.drop(columns=colunas_existentes)

# Excluir dados de fora do Brasil
data = data[(data['lat'] >= -33) & (data['lat'] <= 5) &  # Filtrar latitudes dentro do Brasil
            (data['lon'] >= -74) & (data['lon'] <= -35)]  # Filtrar longitudes dentro do Brasil

# Converter a coluna de 'time' para o formato datetime
data['time'] = pd.to_datetime(data['time'])

# Converter T2M de Kelvin para Celsius
data['T2M_Celsius'] = (data['T2M'] - 273.15).round(2)

# QV2M em %
data['QV2M_percentage'] = (data['QV2M'] * 100).round(3)

# Criar novas variáveis a partir da data
data['year'] = data['time'].dt.year
data['month'] = data['time'].dt.month

# Adicionar margem de erro de temperatura
data['T2M_min'] = (data['T2M_Celsius'] - 3).round(2)
data['T2M_max'] = (data['T2M_Celsius'] + 4).round(2)

# ===== CÁLCULO DA UMIDADE RELATIVA (UR%) =====
# Fórmulas baseadas em QV2M (umidade específica), PS (pressão) e T2M_Celsius (temperatura)

# Pressão de vapor real (e)
data['e'] = (data['QV2M'] * data['PS']) / (0.622 + 0.378 * data['QV2M'])

# Pressão de saturação (es)
data['es'] = 610.78 * np.exp((17.27 * data['T2M_Celsius']) / (data['T2M_Celsius'] + 237.3))

# Umidade relativa em porcentagem
data['UR_percent'] = (data['e'] / data['es'] * 100).clip(0, 100).round(2)

# Remover colunas auxiliares (opcional)
data = data.drop(columns=['e', 'es'])

# Visualizar os dados após o pré-processamento
data.head()

,time,lat,lon,T2M,QV2M,PS,SWGDN,PRECTOT,T2M_Celsius,QV2M_percentage,year,month,T2M_min,T2M_max
65834,2007-01-01 00:30:00,-33.0,-73.750,290.179749,0.010323,101472.945312,356.085449,2.410197e-06,17.030001,1.032296,2007,1,14.030001,21.030001
65835,2007-01-01 00:30:00,-33.0,-73.125,290.286957,0.010436,101435.492188,362.585144,1.699534e-06,17.139999,1.043575,2007,1,14.139999,21.139999
65836,2007-01-01 00:30:00,-33.0,-72.500,290.410400,0.010504,101387.953125,364.396027,1.159539e-06,17.260000,1.050446,2007,1,14.260000,21.260000
65837,2007-01-01 00:30:00,-33.0,-71.875,290.779266,0.010297,100831.757812,362.381805,9.804697e-07,17.629999,1.029742,2007,1,14.629999,21.629999
65838,2007-01-01 00:30:00,-33.0,-71.250,293.430603,0.008232,95709.320312,368.044495,5.514702e-07,20.280001,0.823172,2007,1,17.280001,24.280001


### Testing the Dataset

In this part of the code, we are conducting a simple test to ensure the dataset is consistent by searching for the highest temperature in Celsius. The code finds the maximum value in the `T2M_max` column, which represents the highest temperature recorded, and then checks if this value is reasonable.

This helps verify that the dataset does not contain extreme or unrealistic temperature values, ensuring the data's integrity before further analysis or model training.


In [15]:
# Encontrar a maior temperatura em Celsius
maior_temperatura = data['T2M_max'].max()

# Exibir a maior temperatura
print(f"A maior temperatura registrada foi: {maior_temperatura}°C")

A maior temperatura registrada foi: 39.599998474121094°C


### Performing a Dataset Check

In this section, we are verifying a random period and city (Fortaleza) to check if the data makes sense. The goal is to filter the dataset based on the coordinates of Fortaleza and a specific time period to ensure the data is reasonable.

#### Fortaleza Coordinates:
- **Latitude**: -3.7172
- **Longitude**: -38.5432

A range of 0.25 degrees latitude and 0.3 degrees longitude is applied to capture data near Fortaleza. Additionally, data for January 2020 is selected to check if the recorded temperatures and other parameters make sense for that period.


In [16]:
# Definir as coordenadas de Fortaleza
lat_fortaleza = -3.7172  # Latitude de Fortaleza
lon_fortaleza = -38.5432  # Longitude de Fortaleza

# Expandir a faixa de proximidade de Fortaleza (0.3 graus)
lat_range = 0.25
lon_range = 0.3

# Filtrar dados para coordenadas próximas de Fortaleza (e janeiro de 2012)
dados_fortaleza = data[(data['lat'] >= lat_fortaleza - lat_range) & 
                       (data['lat'] <= lat_fortaleza + lat_range) & 
                       (data['lon'] >= lon_fortaleza - lon_range) & 
                       (data['lon'] <= lon_fortaleza + lon_range) & 
                       (data['time'].dt.year == 2020) & 
                       (data['time'].dt.month == 1)]

# Verificar os dados filtrados
dados_fortaleza.head()


,time,lat,lon,T2M,QV2M,PS,SWGDN,PRECTOT,T2M_Celsius,QV2M_percentage,year,month,T2M_min,T2M_max
32537890,2020-01-01 00:30:00,-3.5,-38.75,301.078247,0.018819,100873.632812,218.136917,0.00005,27.93,1.881876,2020,1,24.93,31.93


### Comparing with Meteomatics Data

In this section, we compare the dataset with data from Meteomatics to verify its accuracy. The average temperature from the Meteomatics graph is 27.93°C, which matches the average temperature in the imported dataset (27.93°C). This comparison confirms that the dataset is accurate and reliable.

We now display the Meteomatics graph for visual inspection.

In [5]:
from IPython.display import SVG, display

# Caminho do arquivo SVG
svg_file_path = 'C:\\Users\\grazi\\Documents\\NASA SPACE APPS\\Notebook\\model\\METEOMATICS_DATA\chartFortaleza.svg'

# Exibir o SVG
display(SVG(svg_file_path))


### Defining Dependent and Independent Variables

In this section, we define the dependent and independent variables for training the model:

In [17]:
# Variáveis independentes (excluindo a coluna de data)
X = data[['lat', 'lon', 'month', 'year']]

# Variáveis dependentes (precipitação, temperatura)
y = data[['T2M_Celsius' ,'PRECTOT']]

# Verificar o tamanho dos dados
X.shape, y.shape


((1086624, 4), (1086624, 2))

### Splitting the Data

In this section, we split the dataset into training and testing sets. This is an important step to ensure that the model is able to learn from one subset of the data (training) and be evaluated on another, unseen subset (testing). We define the following proportions:

- **Training Data**: 80% of the data will be used for training the model.
- **Testing Data**: 20% of the data will be used for testing and evaluating the model.

The split is performed using `train_test_split` from `sklearn.model_selection`. The `random_state` is set for reproducibility, ensuring that the data split is consistent across runs.


In [18]:
# Dividir os dados em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificar o tamanho das divisões
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((869299, 4), (217325, 4), (869299, 2), (217325, 2))

### Data Normalization

In this section, we normalize the data to ensure that all features are on the same scale. This step is important for models that are sensitive to the scale of the input features, such as linear regression, support vector machines, and neural networks.

The **StandardScaler** from `sklearn.preprocessing` is used for normalization. It standardizes the data by removing the mean and scaling to unit variance. 

- The **`fit_transform`** method is applied to the training data to learn the scaling parameters (mean and standard deviation).
- The **`transform`** method is then applied to both the training and testing data to ensure consistency between them.

```python

In [19]:
# Normalizar os dados 
scaler = StandardScaler()

# Ajustar o scaler nos dados de treino e transformar tanto os dados de treino quanto os de teste
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Verificar as primeiras linhas dos dados normalizados
X_train_scaled[:5]

array([[ 1.48418807,  0.05562076, -0.41533375, -0.15588779],
       [ 0.53962586, -0.87903829,  1.0388446 ,  0.95704034],
       [-0.18004058, -1.04397812,  0.45717326,  1.14252836],
       [ 0.76452163,  1.43011935,  1.62051593,  1.32801638],
       [-1.1246028 , -0.98899818,  1.0388446 , -0.71235185]])

### Model Building

In this section, we build and train a model using **LightGBM** (Light Gradient Boosting Machine), which is well-suited for regression tasks and time series data. The model is configured with specific parameters based on the documentation to optimize its performance. The goal is to predict the temperature and precipitation data.

We are using a **MultiOutputRegressor** from `sklearn.multioutput`, which allows us to predict multiple target variables (temperature and precipitation) simultaneously.

The steps include:
1. **Model configuration**: We configure the LightGBM regressor with parameters that control the complexity and performance.
2. **Training the model**: We train the model using the scaled training data (`X_train_scaled` and `y_train`).
3. **Prediction**: We use the trained model to predict values on the test dataset (`X_test_scaled`).
4. **Evaluation**: We evaluate the model using common regression metrics: MAE, MSE, and R².

The results obtained were:
- **MAE (Mean Absolute Error)**: 0.1708
- **MSE (Mean Squared Error)**: 0.1098
- **R² (Coefficient of Determination)**: 0.9213

These metrics indicate excellent prediction performance.


In [20]:
model = lgb.LGBMRegressor(
num_leaves=400,
max_depth=10,
min_data_in_leaf=20,
min_gain_to_split=0,
bagging_fraction = 0.9,
bagging_freq = 10,
learning_rate=0.7
)

# Criar o modelo de regressão multi-output
multi_target_model = MultiOutputRegressor(model)

# Treinar o modelo com X_train_scaled e y_train
multi_target_model.fit(X_train_scaled, y_train)

# Prever os valores de temperatura no conjunto de teste
y_pred = multi_target_model.predict(X_test_scaled)

# Avaliar o desempenho do modelo com métricas de regressão
print(f'MAE (Erro Absoluto Médio): {mean_absolute_error(y_test, y_pred):.4f}')
print(f'MSE (Erro Quadrático Médio): {mean_squared_error(y_test, y_pred):.4f}')\

print(f'R² (Coeficiente de Determinação): {r2_score(y_test, y_pred):.4f}')

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_gain_to_split is set=0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_gain_to_split is set=0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the 

C:\Users\grazi\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_gain_to_split is set=0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10


C:\Users\grazi\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_gain_to_split is set=0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
MAE (Erro Absoluto Médio): 0.1708
MSE (Erro Quadrático Médio): 0.1098
R² (Coeficiente de Determinação): 0.9213


### Exporting the Models

In this section, we export the trained models and the scaler to be used for future predictions. The models and scaler are saved using **`joblib`**, which is a convenient method for serializing Python objects, especially large models like LightGBM.

Steps:
1. **Create the directory**: If it doesn't exist, we create a directory named `model` to store the exported models.
2. **Save the models**: We use `joblib.dump()` to save the trained **MultiOutputRegressor model** and the **scaler**.

This allows us to load and use the models at a later time without needing to retrain them.

In [22]:
# Criar o diretório se ele não existir
os.makedirs('model', exist_ok=True)

# Salvar o modelo treinado
joblib.dump(multi_target_model, 'model/clima_model.joblib')
joblib.dump(scaler, 'model/scaler.joblib')

['model/scaler.joblib']

### Making a Prediction with the Model

In this section, we test the trained model by making a prediction for a new data point. We provide the model with a set of input features (latitude, longitude, month, and year) and use the trained model to predict the temperature and precipitation for that specific location and time.

Steps:
1. **New Data**: We create a new data point with the same features as those used for training, including latitude, longitude, month, and year.
2. **Normalize the Input Data**: We normalize the new data using the same scaler that was used during training to ensure consistency.
3. **Make Predictions**: We use the trained model to predict both temperature and precipitation for the given input data.
4. **Convert Precipitation**: The predicted precipitation value is then multiplied by the number of seconds in 30 days (approximating one month) to convert it to a more tangible unit (mm).
5. **Determine the Chance of Rain**: Based on the converted precipitation value, we classify the chance of rain into one of the following categories:
   - **Low Chance**: Less than 2 mm
   - **Medium Chance**: Between 2 mm and 10 mm
   - **High Chance**: Between 10 mm and 50 mm
   - **Very High Chance**: More than 50 mm
6. **Display the Results**: The model's predictions for temperature and precipitation are displayed, along with the chance of rain based on the converted precipitation.

In [21]:
# Novo dado, com as mesmas variáveis usadas no treino
new_data = pd.DataFrame({
    'lat': [-23.4333],
    'lon': [-45.0833],
    'month': [1],
    'year': [2026]
})

# Normalizar os dados de entrada (mesmo scaler usado no treino) 
new_data_scaled = scaler.transform(new_data)

# Prever o clima com o modelo (incluindo pressão e radiação solar) 
prediction = multi_target_model.predict(new_data_scaled)

# Definir o número de segundos em 30 dias (1 mês)
seconds_in_month = 30 * 24 * 60 * 60

# Multiplicar PRECTOT pela quantidade de segundos no mês
precipitação_convertida = prediction[0][1] * seconds_in_month

# Definir os limites para a chance de chuva
baixa_chuva = 2  # 2 mm, baixa chance
media_chuva = 10  # 10 mm, média chance
alta_chuva = 50  # 50 mm, alta chance
muito_alta_chuva = 50  # Acima de 50 mm, muito alta chance

# Verificar a chance de chuva com base no valor de precipitação convertida
if precipitação_convertida > muito_alta_chuva:
    chance_chuva = "Muito Alta"
elif precipitação_convertida >= alta_chuva:
    chance_chuva = "Alta"
elif precipitação_convertida >= media_chuva:
    chance_chuva = "Média"
else:
    chance_chuva = "Baixa"

# Exibir a precipitação convertida e a chance de chuva
print(f'Temperatura: {round(prediction[0][0], 2)} °C (T2M_Celsius)')
print(f'Precipitação convertida: {round(precipitação_convertida, 2)} mm')
print(f'Chance de chuva: {chance_chuva}')


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_gain_to_split is set=0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_gain_to_split is set=0, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=0
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=10, subsample_freq=0 will be ignored. Current value: bagging_freq=10
Temperatura: 24.92 °C (T2M_Celsius)
Precipitação convertida: 2

C:\Users\grazi\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
C:\Users\grazi\anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


### Comparing with the Meteomatics Forecast

In this section, we compare the predictions from our model with those from **Meteomatics** for the same location, **Ubatuba**. The Meteomatics forecast predicts an average temperature of **24.85°C**, while our model predicts **24.92°C**.

This small difference indicates that our model is making accurate predictions, with the temperature forecast being very close to the one provided by Meteomatics.

We display the **Meteomatics chart** for visual verification of the data:

In [12]:
from IPython.display import SVG, display

# Caminho do arquivo SVG
svg_file_path = 'C:\\Users\\grazi\\Documents\\NASA SPACE APPS\\Notebook\\model\\METEOMATICS_DATA\chartUbatuba.svg'

# Exibir o SVG
display(SVG(svg_file_path))